In [ ]:
import os
import sqlite3
import math
from typing import Dict, List, Union, Any, Optional, Set, Tuple
import re, time
import pandas as pd
import numpy as np

# ===== CONFIG =====
MAGI_DB_PATH = "/projects/klybarge/pcori_ad/magi/magi_db/magi.db"  # read-only
OUT_DIR = "./MAGI_LASSO"
os.makedirs(OUT_DIR, exist_ok=True)
EDGE_TABLE = "magi_counts_top500"   
uri = f"file:{MAGI_DB_PATH}?mode=ro"

TOP_K = 500  # 500 for others

TARGETS = [
    "aa_meas_citalopram_rem",
]

############### MAGI FUNCTION ###############
"""
MAGI: Dependent Bayes with Temporal Ordering — Reference Implementation
-------------------------------------------------------------------------------
This file adds **step-by-step comments** to the function `analyze_causal_sequence_py`. The comments mirror the proposed
algorithm sections:

1) Determination of Temporal Order
2) Estimation of Dependent Bayes (T-values, λ-links, and D-values)
3) Logistic link to produce P(Y=1 | Z)

INPUT TABLE EXPECTATIONS (long format, one row per directed pair):
- target_concept_code : str (# left node in the edge; in many places this
denotes the earlier event)
- concept_code : str (# right node in the edge; the later event)
- n_code_target : float/int (# co-occurrence count for concept & target
in the *target=Y* stratum; used as 'a')
- n_code_no_target : float/int (# co-occurrence count for concept & target
in the *target=¬Y* stratum; used as 'b')
- n_target : float/int (# total Y count for this target concept, per edge row; we take max within a block)
- n_no_target : float/int (# total ¬Y count; max within a block)
- no_code_no_target : float/int (# optional, computed if missing as n_no_target - n_code_no_target; clipped ≥0)
- n_target_before_code: float/int (# count where target occurred before code)
- n_code_before_target: float/int (# count where code occurred before target)

NOTES ON TEMPORAL COUNTS:
For a row with target_concept_code = A and concept_code = B, the columns
`n_code_before_target` and `n_target_before_code` are interpreted as:
- n_code_before_target: # of persons where **B happened before A**
- n_target_before_code: # of persons where **A happened before B**
We aggregate these across j≠i to compute *temporal scores* for each event.

PIECEWISE, SAMPLE-SIZE–ANCHORED ADJUSTMENTS:
- For odds terms that would be 0 or ∞ due to zero cells, we replace the
offending odds with 1/(N+1) or (N+1)/1, where N is the size of the
appropriate stratum, so all ratios remain finite and interpretable.

RETURN VALUE:
A dict with temporal ordering, T-values, λ-vectors, D-values, coefficients
for a logistic link, a `predict_proba` callable, and trace tables.
"""

def analyze_causal_sequence_py(
    data: Union[str, pd.DataFrame],
    name_map: Dict[str, str],
    events: List[str],
    force_outcome=None,
) -> Dict[str, Any]:
    """Compute temporal order, dependent-Bayes direct effects (D), and
    a logistic-link probability for outcome Y from pairwise counts.

    Parameters
    ----------
    data : str | DataFrame
        CSV path or in-memory DataFrame with the columns described above.
    name_map : Dict[str, str]
        Optional mapping raw code -> friendly label. If provided, both
        `target_concept_code` and `concept_code` are replaced.
    events : List[str]
        List of event names/codes to restrict the analysis to. If `None`,
        events are auto-detected from the table and intersected.
    force_outcome : str | None
        If provided and found among events, this event is forced to be the
        **final** node (i.e., the outcome) in the temporal order.

    Returns
    -------
    Dict[str, Any]
        - sorted_scores : pd.Series of temporal scores (desc)
        - temporal_order: list of events (outcome at the end)
        - order_used    : same as temporal_order
        - T_val         : pd.Series of total effects T_{k,Y}
        - D_val         : pd.Series of direct effects D_{k,Y}
        - coef_df       : pd.DataFrame of coefficients (β_k and intercept)
        - lambda_l      : dict[str -> pd.Series] of λ_{k,j} vectors
        - trace_df      : pd.DataFrame detailing the backward recursion steps
        - invalid_predictors: list of predictors whose log(D) was invalid
        - beta_0, beta, logit_predictors, predict_proba: logistic elements
    """

    # ---------------------------------------------------------------------
    # 0) INGEST & BASIC VALIDATION
    # ---------------------------------------------------------------------
    if isinstance(data, str):
        # Read from CSV path
        df = pd.read_csv(data)
    else:
        # Work on a copy to avoid mutating caller's object
        df = data.copy()

    # Ensure required identifier columns are present
    for col in ["target_concept_code", "concept_code"]:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Optional recoding to human-friendly labels
    if name_map:
        df["target_concept_code"] = df["target_concept_code"].replace(name_map)
        df["concept_code"] = df["concept_code"].replace(name_map)

    # Numeric columns the algorithm expects
    need = [
        "n_code_target", "n_code_no_target", "n_target", "n_no_target",
        "n_target_before_code", "n_code_before_target",
    ]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {', '.join(missing)}")

    # If precomputed total_effect exists (for λ or sanity checks), coerce to numeric
    has_total = "total_effect" in df.columns
    if has_total:
        df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")

    # ---------------------------------------------------------------------
    # 1) EVENT SET (optional auto-detect) & TYPE COERCION
    # ---------------------------------------------------------------------
    if events is None:
        # Auto-detect: intersect events appearing on both sides of edges
        ev_targets = df["target_concept_code"].dropna().astype(str).unique().tolist()
        ev_code = df["concept_code"].dropna().astype(str).unique().tolist()
        events = sorted(set(ev_targets).intersection(ev_code))
        if len(events) == 0:
            # Fall back to union if intersection is empty
            events = sorted(set(ev_targets) | set(ev_code))
    if len(events) < 2:
        raise ValueError("Need at least two events after auto-detection.")

    # Keep only rows whose endpoints are both in the chosen event set
    df = df[df["target_concept_code"].isin(events) & df["concept_code"].isin(events)].copy()

    # Coerce numeric columns robustly (invalid -> NaN); subsequent ops handle NaNs
    for c in need:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # If `no_code_no_target` missing, derive as (n_no_target - n_code_no_target) ≥ 0
    if "no_code_no_target" not in df.columns:
        df["no_code_no_target"] = (df["n_no_target"] - df["n_code_no_target"]).clip(lower=0)
    else:
        df["no_code_no_target"] = pd.to_numeric(df["no_code_no_target"], errors="coerce").clip(lower=0)

    # Helper: total count for an event (max n_target where that event is target)
    # This mirrors your original choice; change to .sum() if warranted.
    def C_of(ev: str) -> float:
        sub = df[df["target_concept_code"] == ev]
        if sub.empty:
            return float("nan")
        C = pd.to_numeric(sub["n_target"], errors="coerce").max()
        return float(C) if pd.notna(C) and np.isfinite(C) else float("nan")

    # Helper: numerically safe log (log(1)=0 for invalid/≤0)
    def safe_log(x: float) -> float:
        try:
            xv = float(x)
        except (TypeError, ValueError):
            return 0.0
        if not np.isfinite(xv) or xv <= 0.0:
            return 0.0
        return math.log(xv)

    # ---------------------------------------------------------------------
    # 2) TEMPORAL ORDER — pairwise-before counts → per-node score
    # ---------------------------------------------------------------------
    # Score(Z_i) = Σ_{j≠i} [ C(Z_i≪Z_j) - C(Z_j≪Z_i) + C(Z_i ∩ ¬Z_j) - C(Z_j ∩ ¬Z_i) ]
    # Here we implement the *before/after* portion using the provided columns.
    # If only presence/absence terms are available, you may approximate using
    # the last two terms.
    scores: Dict[str, float] = {}
    for zk in events:
        s = 0.0
        for zj in [x for x in events if x != zk]:
            # For the pair (zj, zk), we interpret:
            #   n_code_before_target   — # where zk (code) before zj (target)
            #   n_target_before_code   — # where zj (target) before zk (code)
            rowr = df[(df["target_concept_code"] == zj) & (df["concept_code"] == zk)]
            if not rowr.empty:
                s += float(rowr["n_code_before_target"].sum(skipna=True) -
                           rowr["n_target_before_code"].sum(skipna=True))
        scores[zk] = s

    sorted_scores = pd.Series(scores).sort_values(ascending=False)

    # Outcome selection:
    #  - If `force_outcome` is provided and present, put it at the end.
    #  - Else, default to the top-scoring node as outcome.
    if force_outcome and (force_outcome in sorted_scores.index):
        outcome_event = force_outcome
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome_event] + [outcome_event]
    else:
        outcome_event = sorted_scores.index[0]
        temporal_order = [ev for ev in sorted_scores.index if ev != outcome_event] + [outcome_event]

    # Propagation order is the temporal order; last is outcome Y
    events_order = temporal_order
    outcome = events_order[-1]
    antecedents = events_order[:-1]

    # ---------------------------------------------------------------------
    # 3) T-VALUES (TOTAL EFFECTS) & λ-LINKS BETWEEN ANTECEDENTS
    # ---------------------------------------------------------------------
    # For each antecedent k, we compute T_{k,Y} as an odds ratio between
    # strata Z_k=1 and Z_k=0 with sample-size–anchored fixes for zero cells.
    # For λ_{k,j} (dependence of j given k), we first use precomputed
    # `total_effect` if present.
    # If λ is missing (no recorded dependence), assume independence
    # and treat the conditional contribution as 0 in the adjustment sum.

    T_val = pd.Series(0.0, index=antecedents, dtype=float)  # T_{k,Y}
    D_val = pd.Series(np.nan, index=antecedents, dtype=float)  # D_{k,Y}
    lambda_l: Dict[str, pd.Series] = {}  # per-k vector of λ_{k,j} for j after k and before Y

    for k in antecedents:
        # ---- Contingency for (k, outcome) ----
        row_ko = df[(df["target_concept_code"] == k) & (df["concept_code"] == outcome)]

        # a: Y∩k    b: ¬Y∩k    c: Y∩¬k    d: ¬Y∩¬k
        a = float(row_ko["n_code_target"].sum(skipna=True))            # co-occurrence in Y
        b = float(row_ko["n_code_no_target"].sum(skipna=True))         # co-occurrence in ¬Y

        # If n_target_no_code absent, approximate c by (max n_target - a)
        if "n_target_no_code" in row_ko.columns:
            c = float(row_ko["n_target_no_code"].sum(skipna=True))
        else:
            c = float(pd.to_numeric(row_ko["n_target"], errors="coerce").max() - a)

        # If no_code_no_target absent, approximate d by (max n_no_target - b)
        if "no_code_no_target" in row_ko.columns:
            d = float(row_ko["no_code_no_target"].sum(skipna=True))
        else:
            d = float(pd.to_numeric(row_ko["n_no_target"], errors="coerce").max() - b)

        N1, N0 = a + b, c + d  # stratum sizes for Z_k=1 and Z_k=0

        # ---- Sample-size–anchored odds for Z_k=1 ----
        if a == 0:
            odds_pos_adj = 1.0 / (N1 + 1.0)  # 0 → tiny positive odds
        elif b == 0:
            odds_pos_adj = (N1 + 1.0) / 1.0  # ∞ → capped by (N1+1)
        else:
            odds_pos_adj = a / b

        # ---- Sample-size–anchored odds for Z_k=0 ----
        if c == 0:
            odds_neg_adj = 1.0 / (N0 + 1.0)
        elif d == 0:
            odds_neg_adj = (N0 + 1.0) / 1.0
        else:
            odds_neg_adj = c / d

        # Total effect T_{k,Y}
        T_val.loc[k] = float(odds_pos_adj / odds_neg_adj)

        # ---- λ_{k,j}: dependence of j on k for nodes j after k (and before Y) ----
        pos_k = events_order.index(k)
        js = events_order[pos_k + 1 : -1] if pos_k < len(events_order) - 1 else []

        lam_pairs = []
        for j in js:
            row_kj = df[(df["target_concept_code"] == k) & (df["concept_code"] == j)]
            if row_kj.empty:
                lam_pairs.append((j, 0.0))  # no evidence of dependence
                continue

            # Otherwise approximate λ with piecewise, size-anchored logic
            # C11 = C(j∩k); Cj_not_k = C(j∩¬k); Ck = C(k)
            C11 = float(row_kj["n_code_target"].sum(skipna=True))  # re-using the same column name for co-occurrence
            if "n_code_no_target" in row_kj.columns:
                Cj_not_k = float(row_kj["n_code_no_target"].sum(skipna=True))
            else:
                Cj = C_of(j)
                Cj_not_k = 0.0 if (not np.isfinite(Cj)) else max(Cj - C11, 0.0)
            Ck = C_of(k)

            if Cj_not_k == 0:
                L = 1.0 + C11          # always-with-k → boost
            elif C11 == 0:
                L = 1.0 / (1.0 + Cj_not_k)  # never-with-k → downweight
            elif np.isfinite(Ck) and Ck > 0:
                L = C11 / Ck           # normalized co-occurrence
            else:
                L = 0.0

            lam_pairs.append((j, float(L)))

        lambda_l[k] = pd.Series({j: v for j, v in lam_pairs}, dtype=float)

    # ---------------------------------------------------------------------
    # 4) BACKWARD RECURSION — resolve D_{k,Y} from T and λ
    # ---------------------------------------------------------------------
    # D_{k,Y} = ( T_{k,Y} - Σ_i λ_{k,k+i} * D_{k+i,Y} ) / ( 1 - Σ_i λ_{k,k+i} )
    # Start at the last antecedent (just before Y): D := T, since there are no
    # downstream nodes to adjust for.

    trace_rows = []  # for human-auditable tracing of the recursion

    last_anc = antecedents[-1] if antecedents else None
    if last_anc is not None:
        D_val.loc[last_anc] = T_val.loc[last_anc]
        trace_rows.append({
            "stage": "Last 2 Nodes",
            "nodes": f"{last_anc} - {outcome}",
            "k": last_anc,
            "T_kY": T_val.loc[last_anc],
            "lambda_terms": None,
            "sum_lambda": 0.0,
            "D_kY": D_val.loc[last_anc],
            "log_D": safe_log(D_val.loc[last_anc]),
        })

    if len(antecedents) > 1:
        # Walk backwards through the remaining antecedents
        for k in list(reversed(antecedents))[1:]:
            lam_vec = lambda_l.get(k, pd.Series(dtype=float))
            # When computing the adjustment, use **only** D-values for nodes that
            # are after k and already resolved (present in lam_vec index).
            code = list(lam_vec.index)
            num = T_val.loc[k] - float(np.nansum(lam_vec.reindex(code).values * D_val.reindex(code).values))
            den = 1.0 - float(np.nansum(lam_vec.values))  # may approach 0 if λ's are large

            # If den is pathological (≤0 or NaN), fall back to T (neutralization).
            D_val.loc[k] = (num / den) if np.isfinite(num / den) else T_val.loc[k]

            span = len(events_order) - events_order.index(k) + 1
            lam_str = ", ".join(
                f"λ_{events_order.index(k)+1}{events_order.index(c)+1}={lam_vec[c]:.6f}"
                for c in code
            ) if len(lam_vec) else None

            trace_rows.append({
                "stage": f"Last {span} Nodes",
                "nodes": " - ".join([k] + events_order[events_order.index(k)+1:]),
                "k": k,
                "T_kY": T_val.loc[k],
                "lambda_terms": lam_str,
                "sum_lambda": float(np.nansum(lam_vec.values)),
                "D_kY": D_val.loc[k],
                "log_D": safe_log(D_val.loc[k]),
            })

    trace_df = pd.DataFrame(trace_rows)

    # ---------------------------------------------------------------------
    # 5) COEFFICIENTS — map D's onto a logistic link
    # ---------------------------------------------------------------------
    # We model:  logit P(Y=1 | Z) = β0 + Σ_k β_k * Z_k,  with β_k = log D_{k,Y}
    # Intercept β0 is set by the marginal prevalence of Y (from `n_target` &
    # `n_no_target`) for the outcome rows.

    resp_rows = df[df["target_concept_code"] == outcome]
    if resp_rows.empty:
        raise ValueError(f"No rows for outcome '{outcome}'.")

    n_t = resp_rows["n_target"].dropna().iloc[0] if resp_rows["n_target"].dropna().size else np.nan
    n_n = resp_rows["n_no_target"].dropna().iloc[0] if resp_rows["n_no_target"].dropna().size else np.nan
    denom = n_t + n_n
    p_y = 0.5 if (not np.isfinite(denom) or denom <= 0) else (n_t / denom)
    beta_0 = float(np.log(p_y / (1 - p_y)))  # stable enough for p∈(0,1)

    # β_k = log(D_{k,Y}); protect against non-positive D by mapping to 0
    D_clean = pd.to_numeric(D_val, errors="coerce").astype(float)
    D_pos = D_clean.where(D_clean > 0)

    with np.errstate(divide="ignore", invalid="ignore"):
        beta_vals = np.log(D_pos.to_numpy())
    beta_k_raw = pd.Series(beta_vals, index=D_val.index)
    invalid_predictors = list(beta_k_raw[~np.isfinite(beta_k_raw)].index)

    beta_k = beta_k_raw.copy()
    beta_k[~np.isfinite(beta_k)] = 0.0  # neutralize invalid predictors

    coef_df = pd.DataFrame({
        "predictor": list(beta_k.index) + ["(intercept)"],
        "beta": list(beta_k.astype(float).values) + [beta_0],
    })

    # ---------------------------------------------------------------------
    # 6) PREDICT_PROBA — vectorized logistic link
    # ---------------------------------------------------------------------
    predictors = list(beta_k.index)
    beta_vec = beta_k.astype(float).values

    def predict_proba(z: Union[Dict[str, Any], pd.Series, np.ndarray, List[float], pd.DataFrame]) -> Union[float, np.ndarray, pd.Series]:
        """Compute P(Y=1 | Z) using the logistic link.

        Accepts:
          - dict/Series mapping predictor name -> 0/1
          - 1D/2D numpy/list with columns ordered as `predictors`
          - DataFrame with columns containing any/all of `predictors` (others ignored)

        Returns:
          - float for 1D inputs; np.ndarray or pd.Series for vectorized inputs
        """
        if isinstance(z, pd.DataFrame):
            Z = z.reindex(columns=predictors, fill_value=0).astype(float).to_numpy()
            eta = beta_0 + Z @ beta_vec
            # Stable sigmoid via clipping; avoids overflow for extreme η
            return 1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700)))

        if isinstance(z, (dict, pd.Series)):
            v = np.array([float(z.get(p, 0.0)) for p in predictors], dtype=float)
            eta = beta_0 + float(v @ beta_vec)
            return float(1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700))))

        arr = np.asarray(z, dtype=float)
        if arr.ndim == 1:
            if arr.size != len(predictors):
                raise ValueError(f"Expected {len(predictors)} features in order: {predictors}")
            eta = beta_0 + float(arr @ beta_vec)
            return float(1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700))))
        else:
            if arr.shape[1] != len(predictors):
                raise ValueError(f"Expected shape (*, {len(predictors)}), got {arr.shape}")
            eta = beta_0 + arr @ beta_vec
            return 1.0 / (1.0 + np.exp(-np.clip(eta, -700, 700)))

    # ---------------------------------------------------------------------
    # 7) PACKAGE RESULTS
    # ---------------------------------------------------------------------
    return {
        "sorted_scores": sorted_scores,
        "temporal_order": temporal_order,
        "order_used": events_order,
        "T_val": T_val,
        "D_val": D_val,
        "coef_df": coef_df,
        "lambda_l": lambda_l,
        "trace_df": trace_df,
        "invalid_predictors": invalid_predictors,
        # Logistic link outputs:
        "beta_0": beta_0,
        "beta": pd.Series(beta_vec, index=predictors, dtype=float),
        "logit_predictors": predictors,
        "predict_proba": predict_proba,
    }

# ===== SNOMED descendants support (all descendants)=====
from collections import defaultdict, deque

IS_A = "116680003"
CHAR_TYPES = {
    "inferred": "900000000000011006",  # classification hierarchy
    "stated":   "900000000000010007",
}

SNOMED_REL_FULL_US = "/projects/klybarge/pcori_ad/magi/Test/Test/sct2_Relationship_Full_US1000124_20250901.txt"

def build_is_a_snapshot(full_rel_path: str, characteristic: str = "inferred") -> pd.DataFrame:
    """From a Full RF2 file, return the *current* active IS-A rows."""
    use_cols = [
        "id","effectiveTime","active","moduleId",
        "sourceId","destinationId","relationshipGroup",
        "typeId","characteristicTypeId","modifierId"
    ]
    df = pd.read_csv(full_rel_path, sep="\t", dtype=str, usecols=use_cols)
    df = df[(df["typeId"] == IS_A) & (df["characteristicTypeId"] == CHAR_TYPES[characteristic])]
    df["effectiveTime_num"] = df["effectiveTime"].astype(int)
    idx = df.groupby("id")["effectiveTime_num"].idxmax()
    snap = df.loc[idx]
    snap = snap[snap["active"] == "1"][["sourceId","destinationId"]].reset_index(drop=True)
    return snap

# Build parent→children map once (lazy init so the script still runs if file missing)
__SNAP_REL__ = None
__P2C__ = None
def _ensure_graph():
    global __SNAP_REL__, __P2C__
    if __SNAP_REL__ is None:
        __SNAP_REL__ = build_is_a_snapshot(SNOMED_REL_FULL_US, characteristic="inferred")
    if __P2C__ is None:
        __P2C__ = defaultdict(set)
        for parent, child in zip(__SNAP_REL__["destinationId"], __SNAP_REL__["sourceId"]):
            __P2C__[parent].add(child)

def find_descendants_sct(concept_id: str) -> set:
    """All is-a descendants (children, grandchildren, ...) for a SNOMED conceptId."""
    _ensure_graph()
    out = set()
    q = deque([concept_id])
    while q:
        cur = q.popleft()
        for kid in __P2C__.get(cur, ()):
            if kid not in out:
                out.add(kid)
                q.append(kid)
    return out

def extract_snomed_id(code: str) -> Optional[str]:
    """
    If code looks like 'dx_SNOMED_<digits>' return '<digits>', else None.
    """
    m = re.fullmatch(r"dx_SNOMED_(\d+)", str(code))
    return m.group(1) if m else None

def snomed_aliases_for_outcome(outcome_code: str) -> Tuple[Optional[Set[str]], Dict[str, str]]:
    """
    Given an outcome like 'dx_SNOMED_254645002', return:
      - aliases_codes: {'dx_SNOMED_<id>', ...} including the root itself (or None if not SNOMED)
      - name_map: mapping every alias_code -> outcome_code (canonical)
    """
    root_id = extract_snomed_id(outcome_code)
    if root_id is None:
        return None, {}

    desc = find_descendants_sct(root_id)
    all_ids = {root_id} | set(desc)
    aliases_codes: Set[str] = {f"dx_SNOMED_{sid}" for sid in all_ids}
    name_map: Dict[str, str] = {alias: outcome_code for alias in aliases_codes if alias != outcome_code}
    return aliases_codes, name_map

# ========= helpers =========

def _ensure_derived_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure derived counts (n_target_no_code, no_code_no_target) exist,
    coerce numeric columns safely, and (if absent) compute per-edge total_effect
    using sample-size–anchored odds. Returns a copy.
    """
    out = df.copy()

    # ---- 1) Base numeric columns ----
    base = ["n_code_target", "n_code_no_target", "n_target", "n_no_target"]
    for c in base:
        if c not in out.columns:
            out[c] = 0
        out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0).clip(lower=0).astype(float)

    # ---- 2) Complements (clip to ≥ 0) ----
    # Y & (k=0)
    if "n_target_no_code" not in out.columns:
        out["n_target_no_code"] = (out["n_target"] - out["n_code_target"]).clip(lower=0)
    else:
        out["n_target_no_code"] = pd.to_numeric(out["n_target_no_code"], errors="coerce").fillna(0).clip(lower=0)

    # (k=0) & (Y=0)
    if "no_code_no_target" not in out.columns:
        out["no_code_no_target"] = (out["n_no_target"] - out["n_code_no_target"]).clip(lower=0)
    else:
        out["no_code_no_target"] = pd.to_numeric(out["no_code_no_target"], errors="coerce").fillna(0).clip(lower=0)

    # ---- 3) total_effect (odds_pos / odds_neg) if absent ----
    if "total_effect" not in out.columns:
        a  = out["n_code_target"].astype(float)        # k=1, Y=1
        b  = out["n_code_no_target"].astype(float)     # k=1, Y=0
        c  = out["n_target_no_code"].astype(float)     # k=0, Y=1
        d  = out["no_code_no_target"].astype(float)    # k=0, Y=0

        N1 = a + b
        N0 = c + d

        # Sample-size–anchored odds with neutral fallback when stratum size is zero
        with np.errstate(divide="ignore", invalid="ignore"):
            odds_pos = np.where(
                N1 == 0, 1.0,                              # no data in Z_k=1 → neutral
                np.where(
                    (a > 0) & (b > 0), a / b,
                    np.where(b == 0, N1 + 1.0, 1.0 / (N1 + 1.0))
                )
            )
            odds_neg = np.where(
                N0 == 0, 1.0,                              # no data in Z_k=0 → neutral
                np.where(
                    (c > 0) & (d > 0), c / d,
                    np.where(d == 0, N0 + 1.0, 1.0 / (N0 + 1.0))
                )
            )

            te = odds_pos / odds_neg

        # Clean any residual non-finite values to neutral 1.0
        te = np.where(np.isfinite(te), te, 1.0)
        out["total_effect"] = te.astype(float)

    return out

# TOPK_HI/LO
def _select_top_by_te_unique_k(k_to_T: pd.DataFrame, top_k: int = TOP_K,
                                   hi: float = 1.5) -> pd.DataFrame:
    """
    Balanced selection without fallback:
      - Take up to top_k//2 strongest risk (TE >= hi)
      - Take up to top_k - top_k//2 strongest protective (TE <= 1/hi)
      - If either side is short, do NOT fill from elsewhere; total may be < top_k
      - Return one row per k (target_concept_code), ranked by extremeness
    """
    df = k_to_T.copy()

    # Ensure total_effect numeric
    df["total_effect"] = pd.to_numeric(df["total_effect"], errors="coerce")
    df = df.dropna(subset=["total_effect"]).copy()
    if df.empty:
        print("[SELECT] no rows with total_effect; returning empty selection.")
        return df

    # Extremeness symmetrical around 1
    df["effect_strength"] = np.where(df["total_effect"] >= 1.0,
                                     df["total_effect"],
                                     1.0 / df["total_effect"])

    # One row per k (keep most extreme)
    best_per_k = (df.sort_values("effect_strength", ascending=False)
                    .drop_duplicates(subset=["target_concept_code"], keep="first"))

    HI = hi
    LO = 1.0 / hi

    risk_pool = best_per_k[best_per_k["total_effect"] >= HI].copy()
    prot_pool = best_per_k[best_per_k["total_effect"] <= LO].copy()

    # Target split
    want_risk = top_k // 2
    want_prot = top_k - want_risk

    # Take strongest from each side, capped by availability
    take_risk = min(len(risk_pool), want_risk)
    take_prot = min(len(prot_pool), want_prot)

    sel_risk = risk_pool.nlargest(take_risk, "effect_strength")
    sel_prot = prot_pool.nlargest(take_prot, "effect_strength")

    selected = pd.concat([sel_risk, sel_prot], ignore_index=True)

    print(f"[SELECT] total unique k={len(best_per_k):,}  "
          f"risk={len(risk_pool):,}  prot={len(prot_pool):,}  "
          f"selected(total)={selected['target_concept_code'].nunique()}  "
          f"with: risk={take_risk}, prot={take_prot}")

    return selected.reset_index(drop=True)

def _fetch_k_to_T(conn, target_code: str) -> pd.DataFrame:
    q = f"""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- k (predictor)
             ccn.concept_code AS concept_code           -- T (outcome)
      FROM {EDGE_TABLE} m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE ccn.concept_code = ?
    """
    return pd.read_sql_query(q, conn, params=[target_code])

def _fetch_k_to_T_in(conn, target_codes: List[str]) -> pd.DataFrame:
    """
    Fetch only k→T rows where concept_code IN (target_codes).
    Uses a temp table to avoid SQLite's variable limit.
    """
    tmp_name = "tmp_targets_magi"
    with conn:
        conn.execute(f"DROP TABLE IF EXISTS {tmp_name}")
        conn.execute(f"CREATE TEMP TABLE {tmp_name}(concept_code TEXT)")
        conn.executemany(f"INSERT INTO {tmp_name}(concept_code) VALUES (?)", [(c,) for c in target_codes])

    q = f"""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,   -- k (predictor)
             ccn.concept_code AS concept_code           -- T (outcome)
      FROM {EDGE_TABLE} m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      JOIN {tmp_name} tmp     ON ccn.concept_code         = tmp.concept_code
    """
    return pd.read_sql_query(q, conn)


def _fetch_subgraph_by_targets(conn, events_list):
    ph = ",".join(["?"] * len(events_list))
    q = f"""
      SELECT m.*,
             tcn.concept_code AS target_concept_code,
             ccn.concept_code AS concept_code
      FROM {EDGE_TABLE} m
      JOIN concept_names tcn ON m.target_concept_code_int = tcn.concept_code_int
      JOIN concept_names ccn ON m.concept_code_int        = ccn.concept_code_int
      WHERE tcn.concept_code IN ({ph})
    """
    return pd.read_sql_query(q, conn, params=list(events_list))

import pandas as pd
import os, time, re
from collections import defaultdict

def expand_targets_with_descendants(targets: List[str]) -> Dict[str, Set[str]]:
    out: Dict[str, Set[str]] = {}
    for T in targets:
        aliases, _name_map = snomed_aliases_for_outcome(T)
        out[T] = aliases if aliases is not None else {T}
    return out

def print_full_targets(targets: List[str], *, save_csv: bool = True, out_dir: str = "./"):
    """
    Pretty-print the expanded target definitions and (optionally) save a CSV with all rows:
      root_target, alias_code, alias_conceptId, is_root
    """
    expanded = expand_targets_with_descendants(targets)

    # pretty print to console
    for T, alias_set in expanded.items():
        # count and show a short sample
        n = len(alias_set)
        sample = ", ".join(sorted(list(alias_set))[:10])
        print("=" * 100)
        print(f"[TARGET] {T}")
        print(f"  aliases (including descendants): {n}")
        print(f"  sample: {sample}{' ...' if n > 10 else ''}")

    # optional CSV dump
    if save_csv:
        rows = []
        for T, alias_set in expanded.items():
            root_id = extract_snomed_id(T)
            for alias in sorted(alias_set):
                alias_id = extract_snomed_id(alias)
                rows.append({
                    "root_target": T,
                    "alias_code": alias,
                    "alias_conceptId": alias_id if alias_id else "",
                    "is_root": (alias == T)
                })
        df = pd.DataFrame(rows, columns=["root_target","alias_code","alias_conceptId","is_root"])
        ts = time.strftime("%Y%m%d-%H%M%S")
        os.makedirs(out_dir, exist_ok=True)
        path = os.path.join(out_dir, f"targets_expanded_{ts}.csv")
        df.to_csv(path, index=False)
        print(f"\n[SAVED] Full target expansions → {path}")

    return expanded

# ---- call (run once after TARGETS is set) ----
expanded_targets = print_full_targets(TARGETS, save_csv=True, out_dir=OUT_DIR)

# ========= main loop (remove dup) =========
uri = f"file:{MAGI_DB_PATH}?mode=ro"
with sqlite3.connect(uri, uri=True) as conn:
    for T in TARGETS:
        print("\n" + "=" * 100)
        print(f"[RUN] Target = {T}")

        # Expand SNOMED outcome to include descendants (aliases); else None for non-SNOMED
        aliases_codes, name_map = snomed_aliases_for_outcome(T)

        # 1) k→T rows: allow concept_code == any alias (if SNOMED), else exact match
        if aliases_codes:
            k_to_T = _fetch_k_to_T_in(conn, sorted(aliases_codes))
        else:
            k_to_T = _fetch_k_to_T(conn, T)

        # --- DEBUG: alias breakdown BEFORE canonicalization ---
        if aliases_codes and not k_to_T.empty:
            present_mask = k_to_T["concept_code"].isin(aliases_codes)
            k_to_T_alias = k_to_T.loc[present_mask].copy()

            alias_break = (
                k_to_T_alias
                .groupby("concept_code", as_index=False)
                .agg(
                    rows=("concept_code", "size"),
                    n_code_target=("n_code_target", "sum"),
                    n_code_no_target=("n_code_no_target", "sum"),
                    n_target=("n_target", "sum"),
                    n_no_target=("n_no_target", "sum"),
                )
                .sort_values("rows", ascending=False)
            )

            present_aliases = set(alias_break["concept_code"])
            print(f"[ALIASES] total_possible={len(aliases_codes)}  present_in_db={len(present_aliases)}")
            print(f"[ALIASES] sample_present: {', '.join(list(present_aliases)[:10])}"
                f"{' ...' if len(present_aliases) > 10 else ''}")

            alias_csv = os.path.join(OUT_DIR, f"alias_breakdown_{T}.csv")
            alias_break.to_csv(alias_csv, index=False)
            print(f"[SAVED] Alias breakdown → {alias_csv}")

        # --- Canonicalize concept_code to the root outcome T (name_map replacement) ---
        # This collapses all aliases back to T so MAGI has a single outcome node.
        if not k_to_T.empty and name_map:
            k_to_T["concept_code"] = k_to_T["concept_code"].replace(name_map)

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(k_to_T)
        k_to_T = k_to_T.drop_duplicates()
        after = len(k_to_T)
        if after < before:
            print(f"[DEDUP] k→T exact-row duplicates removed: {before - after}  (kept {after})")

        if k_to_T.empty:
            print(f"[WARN] No k→T rows for {T}; skipping.")
            continue

        # 2) derive totals/effects on the k→T list
        k_to_T = _ensure_derived_cols(k_to_T)

        # 3) select predictors: top-K by total_effect (one row per k)
        sel_rows = _select_top_by_te_unique_k(k_to_T, top_k=TOP_K)
        if sel_rows.empty:
            print(f"[WARN] No predictors selected for {T}; skipping.")
            continue

        selected_k = set(sel_rows["target_concept_code"].astype(str))
        print(f"[SELECT] unique k available={k_to_T['target_concept_code'].nunique():,}  "
              f"selected={len(selected_k):,}")

        # 4) build subgraph among {T} ∪ selected_k
        events_set = selected_k | {T}
        df_trim = _fetch_subgraph_by_targets(conn, sorted(events_set))

        # Allow concept_code to be either events_set OR (if SNOMED) the aliases
        allowed_right = events_set if not aliases_codes else (events_set | aliases_codes)
        df_trim = df_trim[
            df_trim["target_concept_code"].isin(events_set) &
            df_trim["concept_code"].isin(allowed_right)
        ].copy()

        # Canonicalize alias outcomes → T inside the subgraph before analysis
        # --- AUDIT: preserve original alias in subgraph CSV ---
        if not df_trim.empty and name_map:
            df_trim["concept_code_original"] = df_trim["concept_code"]
            df_trim["concept_code"] = df_trim["concept_code"].replace(name_map)

        # --- NEW: drop exact duplicate rows (all columns identical) ---
        before = len(df_trim)
        df_trim = df_trim.drop_duplicates()
        after = len(df_trim)
        if after < before:
            print(f"[DEDUP] subgraph exact-row duplicates removed: {before - after}  (kept {after})")

        # Recompute derived cols (incl. total_effect) on the subgraph
        df_trim = _ensure_derived_cols(df_trim)

        # Reporting (count k→T using aliases if applicable)
        k_to_T_count = (df_trim["concept_code"] == T).sum()
        T_to_j_count = (df_trim["target_concept_code"] == T).sum()
        print(f"[TRIM] rows={len(df_trim):,}  events={len(events_set)}  "
              f"k→T rows={k_to_T_count}  T→j rows={T_to_j_count}")

        # 5) save subgraph (for audit)
        sub_csv = os.path.join(OUT_DIR, f"magi_subgraph_{T}.csv")
        df_trim.to_csv(sub_csv, index=False)
        print(f"[SAVED] Subgraph → {sub_csv}")

        # 6) run MAGI — pass name_map so analysis sees only the canonical outcome T
        try:
            res = analyze_causal_sequence_py(df_trim, events=None, name_map=name_map or None, force_outcome=T)
        except Exception as e:
            print("[ERROR] MAGI failed:", e)
            continue

        # 7) save NON-ZERO coefficients only
        outcome_used = res.get("order_used", [T])[-1]
        coef_df = res["coef_df"].copy()

        # robust: accept 'coef' or 'beta' column name
        coef_col = "coef" if "coef" in coef_df.columns else ("beta" if "beta" in coef_df.columns else None)
        if coef_col is None:
            raise KeyError("Coefficient column not found in coef_df (expected 'coef' or 'beta').")

        # filter to non-zero (tolerance to avoid tiny numerical noise)
        eps = 1e-12
        mask_nz = coef_df[coef_col].astype(float).abs() > eps
        coef_nz = coef_df.loc[mask_nz].copy()

        coef_csv = os.path.join(OUT_DIR, f"magi_coef_{outcome_used}_nonzero.csv")
        coef_nz.to_csv(coef_csv, index=False)

        print(f"[SAVED] Coefficients (non-zero only) → {coef_csv}  "
              f"| kept={len(coef_nz):,} of {len(coef_df):,}  "
              f"| antecedents={len(res.get('order_used', [])) - 1}  "
              f"| used_total_effect={res.get('used_total_effect', True)}")